In [1]:
from generate_dataset import generate_ds, generate_ds_class
generate_ds("train_5_1000.csv",5,1000)
generate_ds_class("train_class_2500_5.csv",5,2500)

In [1]:
import torch
from torch import optim, nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import transforms as T
from torchvision import models, datasets
import torch.nn.functional as F

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from time import sleep

from utils import imshow, show_batch, show_image, show_image_pair, get_accuracy_bin
from models import ContrastiveLoss, EfficientSiemens, SiameseNetworkSimple, EfficientSiemensBinary, EfficientClassification
from generate_dataset import SiameseDataset, ClassificationDataset
from train import trainSiamens, trainBinary, trainClass

/Users/kristinanikolic/opt/anaconda3/envs/CV/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define main data directory
DATA_DIR = 'tiny-imagenet-200-test' # Original images come in shapes of [3,64,64]

# Define training and validation data paths
TRAIN_DIR = os.path.join(DATA_DIR, 'train') 
VALID_DIR = os.path.join(DATA_DIR, 'val')

In [3]:
# Define device to use (CPU or GPU). CUDA = GPU support for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Using: " + str(device))

Using: cpu


In [4]:
# Define transformation sequence for image pre-processing
# If not using pre-trained model, normalize with 0.5, 0.5, 0.5 (mean and SD)
# If using pre-trained ImageNet, normalize with mean=[0.485, 0.456, 0.406], 
# std=[0.229, 0.224, 0.225])

preprocess_transform_pretrain = T.Compose([
                T.Resize(224), # Resize images to 256 x 256
                #T.CenterCrop(224), # Center crop image
                #T.RandomHorizontalFlip(),
                T.ToTensor(),  # Converting cropped images to tensors
                T.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225])
])

simple = T.Compose([T.ToTensor(),  # Converting cropped images to tensors
])

In [5]:
train_data = ClassificationDataset('train_class_2500_5.csv',TRAIN_DIR,preprocess_transform_pretrain)
batch_size = 128

if use_cuda:
    kwargs = {"pin_memory": True, "num_workers": 1}
else:
    kwargs = {}

# Wrap image dataset (defined above) in dataloader 
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, **kwargs)

In [6]:
#show_image(train_dataloader)

In [7]:
# Declare Siamese Network
net = EfficientClassification(5)
net = net.to(device)
# Decalre Loss Function
criterion = ContrastiveLoss()
criterionB = nn.BCEWithLogitsLoss()
criterionC = nn.CrossEntropyLoss()
# Declare Optimizer
optimizer = optim.Adam(net.parameters(), lr=0.0005, weight_decay=False)
sch = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
# Epochs
epochs = 100
tresholds = [0.5, 0.75, 1.0, 1.25, 1.5]

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = trainClass(epochs=epochs, optimizer=optimizer, sch=sch, tresholds=tresholds, \
                     criterion=criterionC, net = net, device=device, train_dataloader=train_dataloader)
torch.save(model.state_dict(), "model.pt")
print("Model Saved Successfully") 

EfficientClassification(
  (model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
            

  0%|          | 0/20 [00:04<?, ?it/s]


IndexError: Target 3 is out of bounds.